In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
pip install wordcloud


In [ ]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from matplotlib import style
style.use('ggplot')
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('amazon_alexa.tsv',sep='\t')
df.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [ ]:
X = df['verified_reviews']
Y = df['feedback']

# Correcting spelling mistakes

In [ ]:
pip install --user textblob

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
pip install spacy

In [ ]:
pip install pyspellchecker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 26.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import spacy
from spellchecker import SpellChecker

nlp = spacy.load("en_core_web_sm")  # Load a small English model
spellchecker = SpellChecker()

def extract_spelling_mistakes_spacy(text):
    """Extracts potential spelling mistakes from a given text using spaCy and spellchecker."""
    if not pd.isna(text):
        doc = nlp(text)
        mistakes = []
        for token in doc:
            if not nlp.vocab[token.text].is_oov:  # Check if the token is in the vocabulary
                continue  # Skip correctly spelled words
            corrected = spellchecker.correction(token.text)
            if corrected != token.text:
                mistakes.append(token.text)
        return mistakes
    else:
        return []

# Assuming you have a large DataFrame
try:
    df['spelling_mistakes'] = df['verified_reviews'].apply(extract_spelling_mistakes_spacy)
except KeyboardInterrupt:
    print("Interrupted! Stopping execution.")

print(df['spelling_mistakes'])


0                                    []
1                                    []
2                            [Alexa,  ]
3                                  [yr]
4                                    []
                     ...               
3145                                 []
3146                                 []
3147    [TV, ..., npr, stat, ...., ...]
3148                [n't, n't, 're, 'd]
3149                                 []
Name: spelling_mistakes, Length: 3150, dtype: object


In [ ]:
from textblob import TextBlob
def texting(data):
    text = [TextBlob(word).correct() for word in data]
    return data


In [ ]:
from textblob import TextBlob

def texting(data):
    if isinstance(data, str):
        text = [TextBlob(word).correct() for word in data.split()]
        return ' '.join(text)
    else:
        return str(data)  # Handle non-string values by converting to string

df['verified_reviews'] = df['verified_reviews'].apply(lambda x: texting(x))



TypeError: sequence item 0: expected str instance, TextBlob found

In [ ]:
df['verified_reviews'] = df['verified_reviews'].apply(lambda x: texting(x))


TypeError: 'float' object is not iterable

In [ ]:
df['verified_reviews']

# Preprocessing Text (lower- tokenizer - regular - Handling Slang and Emoji)


In [ ]:
def data_processing(text):
    text = text.lower()
    text = re.sub(r"http\S+www\S+|https\S+", '', text, flags= re.MULTILINE)
    text = re.sub(r'[^\w\s]','',text) #Emojis are considered special characters and will be removed by the regular expression
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

In [ ]:
df.verified_reviews = df['verified_reviews'].apply(data_processing)


In [ ]:
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data]
    return data

In [ ]:
df['verified_reviews'] = df['verified_reviews'].apply(lambda x: stemming(x))


In [ ]:
for i in range(5):
    print(df['verified_reviews'].iloc[i],"\n")

# Word Embedding


# One Hot Representation


In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
print(df.columns)


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['verified_reviews'])

# Get the vocabulary size
voc_size = len(tokenizer.word_index) + 1

print(voc_size)

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in df['verified_reviews']]
print(onehot_repr)

# vectorization

# 1-BOW -

In [ ]:
cv = CountVectorizer()
X = cv.fit_transform(df['verified_reviews'])
#cv.vocabulary_

# NGRAM(UNI -BI-TRI) (the BOW is special case of n-grams)
NGRAM (SAVE THE ORDER (SEQUENCE))

In [ ]:
cv = CountVectorizer(ngram_range=(1,1))
X_uni = cv.fit_transform(df['verified_reviews'])
#cv.vocabulary_

In [ ]:
cv = CountVectorizer(ngram_range=(1,2))
X_bi = cv.fit_transform(df['verified_reviews'])
#cv.vocabulary_

In [ ]:
cv = CountVectorizer(ngram_range=(1,3))
X_tri = cv.fit_transform(df['verified_reviews'])
#cv.vocabulary_

# TF-IDF (Term Frequency - Inverse Document Frequency)
number for importance-semantic meaning


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
x_tf=tf.fit_transform(df['verified_reviews'])
#tf.vocabulary_
x_tf.toarray()

In [ ]:
import pandas as pd
df=pd.DataFrame(x_tf.toarray(),columns= tf.get_feature_names_out())
df.head()



# Word Embedding Represntation

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd


In [ ]:
#padding
sent_length=15
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)


In [ ]:
dim =50
model=Sequential()
model.add(Embedding(voc_size,50,input_length=sent_length))
model.compile('adam','mse')

In [ ]:
model.summary()

In [ ]:
print(model.predict(embedded_docs))


In [ ]:
embedded_docs[0]


In [ ]:
# ال 0 ده اول خانه والفيكتور بتاعها عباره عن 50  بيعبر عن نسبه اقترابها لكل صف من ال 50
print(model.predict(embedded_docs)[0])


# handle imbalanced datasets

In [ ]:
pos_reviews = df[df.feedback == 1]
pos_reviews.head()

In [ ]:
neg_reviews = df[df.feedback==0]
neg_reviews.head()

In [ ]:
Y.value_counts()


In [ ]:
# Show pie plot (Approach 1)
Y.value_counts().plot.pie(autopct='%.2f')

# Undersampling 'not minority' = resample all classes but the minority class



In [ ]:
pip install --user -U imbalanced-learn


In [ ]:
pip install --upgrade scikit-learn


In [ ]:

from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(sampling_strategy=1) # Numerical value

X_res, y_res = rus.fit_resample(X, Y)

ax = y_res.value_counts().plot.pie(autopct='%.2f')
_ = ax.set_title("Under-sampling")

#  Oversampling "not majority" = resample all classes but the majority class

In [ ]:
from imblearn.over_sampling import RandomOverSampler

#ros = RandomOverSampler(sampling_strategy=1) # Float
ros = RandomOverSampler(sampling_strategy="not majority") # String
X_res, y_res = ros.fit_resample(X, Y)

ax = y_res.value_counts().plot.pie(autopct='%.2f')
_ = ax.set_title("Over-sampling")

In [ ]:
y_res.value_counts()

# # MODELING


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)


# LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
logreg_pred = logreg.predict(x_test)
logreg_acc = accuracy_score(logreg_pred, y_test)
print("Test accuracy: {:.2f}%".format(logreg_acc*100))

In [ ]:
print(confusion_matrix(y_test, logreg_pred))
print("\n")
print(classification_report(y_test, logreg_pred))

In [ ]:
mnb = MultinomialNB()
mnb.fit(x_train, y_train)
mnb_pred = mnb.predict(x_test)
mnb_acc = accuracy_score(mnb_pred, y_test)
print("Test accuracy: {:.2f}%".format(mnb_acc*100))

In [ ]:
print(confusion_matrix(y_test, mnb_pred))
print("\n")
print(classification_report(y_test, mnb_pred))

# KNeighbors

In [ ]:

from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier()

knn_classifier.fit(x_train,y_train)

y_pred_knn = knn_classifier.predict(x_test)

accuracy_knn = accuracy_score(y_test, y_pred_knn)
report_knn = classification_report(y_test, y_pred_knn)

print("Test accuracy: {:.2f}%".format(accuracy_knn*100))

#print("Classification Report:")
#print(report_knn)

In [ ]:
print(confusion_matrix(y_test, y_pred_knn))
print("\n")
print(classification_report(y_test, y_pred_knn))

# Decision tree

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# using a TF-IDF vectorizer

X_train, X_test, y_train, y_test = train_test_split(x_tf,Y, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier()

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy:", accuracy)
print("Test accuracy: {:.2f}%".format(accuracy*100))
# = classification_report(y_test, y_pred)
#print(report_knn)

In [ ]:
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

# Ensemble Learning

# bagging (bootstrap aggregation) | random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score  # For classification

model = RandomForestClassifier(n_estimators=100, random_state=42)  # Instantiates a RandomForestClassifier with 100 trees.

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy: {:.2f}%".format(accuracy*100))
#report_knn = classification_report(y_test, y_pred)
#print(report_knn)

In [ ]:
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

# AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

ada_model = AdaBoostClassifier(n_estimators=100, random_state=42)# n_estimators=100 number of stamb tree

ada_model.fit(X_train, y_train)

y_pred = ada_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy: {:.2f}%".format(accuracy*100))
#report_knn = classification_report(y_test, y_pred)
#print(report_knn)


In [ ]:
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

# Naive Bayes

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from scipy import sparse  # Import for sparse matrix check (optional)

# Assuming X is your feature matrix and Y is your target variable

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Check for sparse matrix and convert if necessary
if isinstance(x_train, sparse.csr_matrix):
    print("x_train is a sparse matrix. Converting to dense array...")
    x_train_dense = x_train.toarray()
    x_train = x_train_dense

if isinstance(x_test, sparse.csr_matrix):
    print("x_test is a sparse matrix. Converting to dense array...")
    x_test_dense = x_test.toarray()
    x_test = x_test_dense  # Update x_test with the dense version

nb_classifier = GaussianNB()

nb_classifier.fit(x_train, y_train)

y_pred = nb_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy: {:.2f}%".format(accuracy*100))
print(classification_report(y_test, y_pred))


# xgboost

In [ ]:
pip install xgboost


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

xgb_model = xgb.XGBClassifier()  # For classification, use XGBClassifier

xgb_model.fit(x_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy: {:.2f}%".format(accuracy*100))
print(classification_report(y_test, y_pred))


# SVC

In [ ]:
import numpy as np  # For numerical operations
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC  # Support Vector Machine classifier
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and testing sets (80% training, 20% testing)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create an SVM classifier with a linear kernel (adjust kernel if needed)
svm_model = SVC(kernel='linear')  # Replace 'linear' with 'poly', 'rbf', etc.

svm_model.fit(x_train, y_train)

y_pred = svm_model.predict(x_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy: {:.2f}%".format(accuracy*100))
print(classification_report(y_test, y_pred))
